In [ ]:
import numpy as np
from matplotlib.pyplot import *
import h5py
sys.path.append('/home/fdfuller/work/analytic_von_hamos/') 
from analytic_von_hamos.raytracing import *
import gpflow
import tensorflow as tf
from analytic_von_hamos.data_extraction import get_peaks_and_intensity as pk
import tensorflow_probability as tfp
tfd = tfp.distributions
from IPython.display import Image, HTML, clear_output
tfb = tfp.bijectors
import imageio
import os
import pickle
%matplotlib inline

In [ ]:
def read_from_pickle(path):
    with open(path, 'rb') as file:
        try:
            while True:
                yield pickle.load(file)
        except EOFError:
            pass
        
def get_2D_from_pickle(path):
    x = next(read_from_pickle(path))
    xaxis = np.linspace(x.xlimits[0], x.xlimits[1], x.total2D.shape[0])
    yaxis = np.linspace(x.ylimits[0], x.ylimits[1], x.total2D.shape[1])
    return x.total2D, xaxis, yaxis

def get_manifold_from_pickle(path, percentile=96):
    rI, rx, ry = get_2D_from_pickle(path)
    f = rI > np.percentile(rI[:],(percentile,))
    X, Y = np.meshgrid(rx,ry)
    return np.stack([Y[f], X[f]],-1)

In [ ]:
class ARaycingModel(gpflow.base.Module, gpflow.models.InternalDataTrainingLossMixin):
# class OneLineFit(gpflow.models.model.GPModel, gpflow.models.InternalDataTrainingLossMixin, Standardizer):
    r"""
    Gaussian Process Regression.
    This is a vanilla implementation of GP regression with a Gaussian
    likelihood.  Multiple columns of Y are treated independently.

    The log likelihood of this model is sometimes referred to as the 'log
    marginal likelihood', and is given by
    .. math::
       \log p(\mathbf y \,|\, \mathbf f) =
            \mathcal N(\mathbf{y} \,|\, 0, \mathbf{K} + \sigma_n \mathbf{I})
    """
    def __init__(
        self,
        energy=None,
        a=None, 
        b=None, 
        c=None,
        p=None, 
        q=None, 
        r=None,
        t=None, 
        u=None, 
        v=None,
        radius=None,
        mm_to_pixels = 20,
        pts_per_iteration = 1000,
        theta_min = -np.pi/2.15,
        theta_max = np.pi/2.15,
        xtal = CrystalSi(4,4,0)
    ):
#         super().__init__()
        
        if energy is None:
            raise ValueError('must specify initial energy')
        if a is None:
            raise ValueError('must specify initial a')
        if b is None:
            raise ValueError('must specify initial b')
        if c is None:
            raise ValueError('must specify initial c')
        if p is None:
            raise ValueError('must specify initial p')
        if q is None:
            raise ValueError('must specify initial q')
        if r is None:
            raise ValueError('must specify initial r')
        if t is None:
            raise ValueError('must specify initial t')
        if u is None:
            raise ValueError('must specify initial u')
        if v is None:
            raise ValueError('must specify initial v')
        if radius is None:
            raise ValueError('must specify initial radius')
            
        self.energy = gpflow.Parameter(energy)
        self.a = gpflow.Parameter(float(a))
        self.b = gpflow.Parameter(float(b))
        self.c = gpflow.Parameter(float(c))
        self.p = gpflow.Parameter(float(p))
        self.q = gpflow.Parameter(float(q))
        self.r = gpflow.Parameter(float(r))    
        self.t = gpflow.Parameter(float(t),transform=tfb.Exp())
        self.u = gpflow.Parameter(float(u))
        self.v = gpflow.Parameter(float(v))
        self.radius = tf.convert_to_tensor(radius,dtype=gpflow.default_float())
        self.xtal = xtal
        self.mm_to_pixels = mm_to_pixels  #convert from mm to det pixel space (20pixels/mm)
        self.theta_min = theta_min
        self.theta_max = theta_max
        self.theta_range = tf.cast(tf.linspace(self.theta_min,
                                       self.theta_max,
                                       pts_per_iteration),tf.float64)
    
    def nan_filter(self, energy):
        y = dety(self.xtal(energy),self.a,self.b,self.c,self.p,self.q,self.r,
                                 self.t,self.u,self.v,self.radius,self.theta_range)
        z = detz(self.xtal(energy),self.a,self.b,self.c,self.p,self.q,self.r,
                                 self.t,self.u,self.v,self.radius,self.theta_range)
        nan_filter_y = tf.math.logical_not(tf.math.is_nan(y))
        nan_filter_z = tf.math.logical_not(tf.math.is_nan(z))
        return tf.math.logical_and(nan_filter_y, nan_filter_z)
    
    def raytracing_pts(self, energy) -> tf.Tensor:   #call realspace_raytracing_predictions
        nan_filter = tf.stop_gradient(self.nan_filter(energy))
        
        y = dety(self.xtal(energy),self.a,self.b,self.c,self.p,self.q,self.r,
                                 self.t,self.u,self.v,self.radius,self.theta_range[nan_filter])
        z = detz(self.xtal(energy),self.a,self.b,self.c,self.p,self.q,self.r,
                                 self.t,self.u,self.v,self.radius,self.theta_range[nan_filter])
        detector_pts = tf.concat([y[:,None], z[:,None]],-1)
        return detector_pts
    
    def __call__(self):
        if len(self.energy.get_shape()) > 0:
            pts = [self.raytracing_pts(energy)[None,:,:] for energy in self.energy]
            pts = tf.concat(pts, 0)
        else:
            pts = self.raytracing_pts(self.energy)
        return pts

In [ ]:
r = 500

parameters1 = {
    'a': 500,
    'b': 0,
    'c': -500,
    'p': 505,
    'q': 0,
    'r': 500,
    't': 1,
    'u': 0,
    'v': 0,
    'radius': r,
    'energy': (9130,9140)
}

parameters2 = {
    'a': 450,
    'b': 0,
    'c': -500,
    'p': 550,
    'q': 0,
    'r': 500,
    't': 1,
    'u': 0,
    'v': 0,
    'radius': r,
    'energy': (9130,9140),
    'theta_max': np.pi/2.02,
    'theta_min': -np.pi/2.02,
}

In [ ]:
m = ARaycingModel(**parameters1)
figure(figsize=(5,5))
predicted_detpts = m()
manifold_1 = get_manifold_from_pickle('../XRT/output_at_p=505_r=500_a=500_c=-500_e=9130.pickle', percentile=99)
manifold_2 = get_manifold_from_pickle('../XRT/output_at_p=505_r=500_a=500_c=-500_e=9140.pickle', percentile=99)
scatter(manifold_1[:,1], manifold_1[:,0],marker='.',c='k',label='xrt simulation at 9130 eV', alpha=0.5)
scatter(manifold_2[:,1], manifold_2[:,0],marker='.',c='g',label='xrt simulation at 9140 eV', alpha=0.5)
plot(predicted_detpts[0,:,1],predicted_detpts[0,:,0],'r-',label='ours at 9130 ev')
plot(predicted_detpts[1,:,1],predicted_detpts[1,:,0],'b-',label='ours at 9140 eV')
legend(loc=5)
xlim([-1,40])
ylim([-30,30])
ax = gca()
ax.set_aspect(3/4)
xlabel('Z (mm)')
ylabel('Y (mm)')
savefig('Geom_agreement_1.pdf')

In [ ]:
m = ARaycingModel(**parameters2)
figure()
predicted_detpts = m()
manifold_1 = get_manifold_from_pickle('../XRT/output_at_p=550_r=500_a=450_c=-500_e=9130.pickle', percentile=99)
manifold_2 = get_manifold_from_pickle('../XRT/output_at_p=550_r=500_a=450_c=-500_e=9140.pickle', percentile=99)
scatter(manifold_1[:,1], manifold_1[:,0],marker='.',c='k',label='xrt simulation at 9130 eV', alpha=0.5)
scatter(manifold_2[:,1], manifold_2[:,0],marker='.',c='g',label='xrt simulation at 9140 eV', alpha=0.5)
plot(predicted_detpts[0,:,1],predicted_detpts[0,:,0],'r-',label='ours at 9130 ev')
plot(predicted_detpts[1,:,1],predicted_detpts[1,:,0],'b-',label='ours at 9140 ev')
legend(loc=5)
xlim([-15,30])
ylim([-10,10])
# xlim([-50,50])
# ylim([-50,50])
ax = gca()
ax.set_aspect(3/4)
xlabel('Z (mm)')
ylabel('Y (mm)')
# savefig('cartoid_optim



# scatter(manifold_2[:,1], manifold_2[:,0],marker='.',c='k',label='xrt simulation')
# plot(predicted_detpts[0,:,1],predicted_detpts[0,:,0],'r-',label='9130 ev')
# plot(predicted_detpts[1,:,1],predicted_detpts[1,:,0],'b-',label='9140 eV')
# legend(loc=6)
# xlim([-15,5])
# ylim([-10,10])
# ax = gca()
# ax.set_aspect(3/4)
# xlabel('Z (mm)')
# ylabel('Y (mm)')
savefig('Geom_agreement_2.pdf')



# m = ARaycingModel(**parameters2)


# figure()
# predicted_detpts = m()
# rI, rx, ry = get_2D_from_pickle('../XRT/output_at_p=550_r=500_a=450_c=-500.pickle')
# contour(rx, ry, rI, 100)
# scatter(predicted_detpts[:,1],predicted_detpts[:,0],c='r',marker='.')

In [ ]:
def change_of_basis(t,u,v):
    if t.ndim == 1:
        t = t[:,None]
    if u.ndim == 1:
        u = u[:,None]
    if v.ndim == 1:
        v = v[:,None]
    tuv = np.concatenate([np.atleast_2d(t),np.atleast_2d(u),np.atleast_2d(v)],-1)
    row_1 = tuv
    row_2 = -np.cross(tuv,np.cross((1,0,0), (0,1,0))[None,:])
    row_3 = np.cross(tuv, np.array((0,1,0))[None,:])
    T = np.concatenate([row_1[:,None,:], row_2[:,None,:], row_3[:,None,:]],1)
    return np.linalg.inv(T)

def initial_guesses_for_experimental_geometry(N: int):
    grand = lambda x0, xu: np.random.uniform(low=x0-xu/2, high=x0+xu/2, size=(N,))
    f = 250
    c0 = -490
    a0 = 500
    b0 = 0.
    d0 = 2*f*np.sqrt(2)
    d_uncertainty = 10
    a_uncertainty = 10
    b_uncertainty = 10
    c_uncertainty = 10
    
    
    def ac_from_dc():
        d = grand(d0,d_uncertainty)
        c = grand(c0,c_uncertainty)
        return np.sqrt(d**2 - c**2), c
    a,c = ac_from_dc()
    b = grand(b0,b_uncertainty)
    
    p0 = 515
    q0 = 0.0
    r0 = 500
    p_uncertainty = 10
    q_uncertainty = 10
    r_uncertainty = 10
    
    dd0 = 700
    p = grand(p0, p_uncertainty)
    r = grand(r0, r_uncertainty)
    q = grand(q0, q_uncertainty)
    
    t0 = 1
    u0 = 0
    v0 = 0
    t_uncertainty = 0.005
    u_uncertainty = 0.005
    v_uncertainty = 0.005
    
    t = grand(t0,t_uncertainty)
    u = grand(u0,u_uncertainty)
    v = grand(v0,v_uncertainty)
    T = change_of_basis(t,u,v)
    
    pqr = np.concatenate([p[:,None],q[:,None],r[:,None]],-1)
    pqr_prime = np.einsum('ijk,ik->ij',T,pqr)
    
    pp = pqr_prime[:,0]
    qp = pqr_prime[:,1]
    rp = pqr_prime[:,2]
    if N == 1:
        return {'a': float(a), 'b': float(b), 'c': float(c),
                'p': float(p), 'q': float(q), 'r': float(r),
                't': float(t), 'u': float(u), 'v': float(v)}
    else:
        return {'a': a, 'b': b, 'c': c, 'p': p, 'q': q, 'r': r, 't': t, 'u': u, 'v': v}
    

In [ ]:
{**initial_guesses_for_experimental_geometry(1), 'energy': 9130, 'radius': 500}

In [ ]:
initial_guess = initial_guesses_for_experimental_geometry(250)
m = TwoLineFit(get_manifold_from_pickle('../XRT/output_at_p=505_r=500_a=500_c=-500_e=9130.pickle', percentile=99),
               get_manifold_from_pickle('../XRT/output_at_p=505_r=500_a=500_c=-500_e=9140.pickle', percentile=99),
               **{**initial_guess,
                                              'radius': 500,
                                              'energy1': 9130,
                                              'energy2': 9140,
                                              'mm_to_pixels': 1.,
                                              'xtal': CrystalSi(4,4,0),
                                              'theta_min': -np.pi/2.2,
                                              'theta_max': np.pi/2.2,
                                             })

opt_i0 = tf.argmin(m._individual_training_losses())
opt_loss0 = m._individual_training_losses()[opt_i0]
p0 = m.get_params(opt_i0)
print('initial best guess loss: ', opt_loss0)

def plot_loss(loss_log, best_loss_log):
    figure(figsize=(10, 4))
    title('Loss history')
    semilogy(loss_log, 'k-.',alpha=0.3)
    figure(figsize=(10, 4))
    title('Best Loss history')
    semilogy(best_loss_log, 'r-.',alpha=0.9)
    show()
    
lr = 1E-2
# lr_sched = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
#     [1000, 2000, 3000, 4000, 5000], [lr, lr*0.5, lr*(0.5**2), lr*(0.5**3), lr*(0.5**4), lr*(0.5**5)])
lr_sched = tf.keras.experimental.CosineDecayRestarts(lr, 1000, alpha=0.1, m_mul=0.9, t_mul=1.0)
trainer = tf.keras.optimizers.Adam(lr_sched)

@tf.function
def train_step():
    with tf.GradientTape() as g:
        loss = m.training_loss()
    grads = g.gradient(loss, m.trainable_variables)
#     grads = [g/(tf.norm(g)+1e-8) for g in grads]
    trainer.apply_gradients(zip(grads, m.trainable_variables))
    return loss

loss_log = []
best_loss_log = []
prediction_log = []
M = 100
for i in range(3000):
    loss = train_step()
    loss_log.append(loss.numpy())
    step_i = len(loss_log)
    if step_i%10 == 0:
        prediction_log.append(m.raytracing_slice(m.energy1, 20))
    if step_i%100 == 0:
        clear_output()
        indiv_losses = m._individual_training_losses()
        best_loss_ind = tf.argmin(indiv_losses)
        best_loss_log.append(indiv_losses[best_loss_ind].numpy().item())
        plot_loss(loss_log, best_loss_log)
    if step_i%150 == 0:
        # prune the bad ones
        s = tf.argsort(m._individual_training_losses()).numpy()
        # replace bad ones with scrambled versions of the top 20 particles
        ap = np.concatenate([m.a.numpy()[s[:-M]], m.a.numpy()[s[np.random.choice(M,M)]]],0)
        bp = np.concatenate([m.b.numpy()[s[:-M]], m.b.numpy()[s[np.random.choice(M,M)]]],0)
        cp = np.concatenate([m.c.numpy()[s[:-M]], m.c.numpy()[s[np.random.choice(M,M)]]],0)
        pp = np.concatenate([m.p.numpy()[s[:-M]], m.p.numpy()[s[np.random.choice(M,M)]]],0)
        qp = np.concatenate([m.q.numpy()[s[:-M]], m.q.numpy()[s[np.random.choice(M,M)]]],0)
        rp = np.concatenate([m.r.numpy()[s[:-M]], m.r.numpy()[s[np.random.choice(M,M)]]],0)
        tp = np.concatenate([m.t.numpy()[s[:-M]], m.t.numpy()[s[np.random.choice(M,M)]]],0)
        up = np.concatenate([m.u.numpy()[s[:-M]], m.u.numpy()[s[np.random.choice(M,M)]]],0)
        vp = np.concatenate([m.v.numpy()[s[:-M]], m.v.numpy()[s[np.random.choice(M,M)]]],0)
        m.a.assign(ap)
        m.b.assign(bp)
        m.c.assign(cp)
        m.p.assign(pp)
        m.q.assign(qp)
        m.r.assign(rp)
        m.t.assign(tp)
        m.u.assign(up)
        m.v.assign(vp)
    print('\r step: %d, loss: %.3f'%(len(loss_log), loss), end='')

In [ ]:
for i in range(9000):
    loss = train_step()
    loss_log.append(loss.numpy())
    step_i = len(loss_log)
    if step_i%10 == 0:
        prediction_log.append(m.raytracing_slice(m.energy1, 20))
    if step_i%100 == 0:
        clear_output()
        indiv_losses = m._individual_training_losses()
        best_loss_ind = tf.argmin(indiv_losses)
        best_loss_log.append(indiv_losses[best_loss_ind].numpy().item())
        plot_loss(loss_log, best_loss_log)
    if step_i%150 == 0:
        # prune the bad ones
        s = tf.argsort(m._individual_training_losses()).numpy()
        # replace bad ones with scrambled versions of the top 20 particles
        ap = np.concatenate([m.a.numpy()[s[:-M]], m.a.numpy()[s[np.random.choice(M,M)]]],0)
        bp = np.concatenate([m.b.numpy()[s[:-M]], m.b.numpy()[s[np.random.choice(M,M)]]],0)
        cp = np.concatenate([m.c.numpy()[s[:-M]], m.c.numpy()[s[np.random.choice(M,M)]]],0)
        pp = np.concatenate([m.p.numpy()[s[:-M]], m.p.numpy()[s[np.random.choice(M,M)]]],0)
        qp = np.concatenate([m.q.numpy()[s[:-M]], m.q.numpy()[s[np.random.choice(M,M)]]],0)
        rp = np.concatenate([m.r.numpy()[s[:-M]], m.r.numpy()[s[np.random.choice(M,M)]]],0)
        tp = np.concatenate([m.t.numpy()[s[:-M]], m.t.numpy()[s[np.random.choice(M,M)]]],0)
        up = np.concatenate([m.u.numpy()[s[:-M]], m.u.numpy()[s[np.random.choice(M,M)]]],0)
        vp = np.concatenate([m.v.numpy()[s[:-M]], m.v.numpy()[s[np.random.choice(M,M)]]],0)
        m.a.assign(ap)
        m.b.assign(bp)
        m.c.assign(cp)
        m.p.assign(pp)
        m.q.assign(qp)
        m.r.assign(rp)
        m.t.assign(tp)
        m.u.assign(up)
        m.v.assign(vp)
    print('\r step: %d, loss: %.3f'%(len(loss_log), loss), end='')

In [ ]:
preds = tf.stack(prediction_log,0)
filenames = []
for k in range(8,300):
    plot(preds[k,0,:,1],preds[k,0,:,0],'r-')
    plot(preds[k,1,:,1],preds[k,1,:,0],'k-', alpha=0.3)
    plot(preds[k,2,:,1],preds[k,2,:,0],'k-', alpha=0.3)
    plot(preds[k,3,:,1],preds[k,3,:,0],'k-', alpha=0.3)
    plot(preds[k,4,:,1],preds[k,4,:,0],'k-', alpha=0.3)
    plot(preds[k,5,:,1],preds[k,5,:,0],'k-', alpha=0.3)
    plot(preds[k,6,:,1],preds[k,6,:,0],'k-', alpha=0.3)
    plot(preds[k,7,:,1],preds[k,7,:,0],'k-', alpha=0.2)
    plot(preds[k,8,:,1],preds[k,8,:,0],'k-', alpha=0.2)
    plot(preds[k,9,:,1],preds[k,9,:,0],'k-', alpha=0.2)
    plot(preds[k,10,:,1],preds[k,10,:,0],'k-', alpha=0.2)
    plot(preds[k,11,:,1],preds[k,11,:,0],'k-', alpha=0.2)
    plot(preds[k,12,:,1],preds[k,12,:,0],'k-', alpha=0.2)
    plot(preds[k,13,:,1],preds[k,13,:,0],'k-', alpha=0.1)
    plot(preds[k,14,:,1],preds[k,14,:,0],'k-', alpha=0.1)
    plot(preds[k,15,:,1],preds[k,15,:,0],'k-', alpha=0.1)
    plot(preds[k,16,:,1],preds[k,16,:,0],'k-', alpha=0.1)
    plot(preds[k,17,:,1],preds[k,17,:,0],'k-', alpha=0.1)
    plot(preds[k,18,:,1],preds[k,18,:,0],'k-', alpha=0.1)
    plot(preds[k,19,:,1],preds[k,19,:,0],'k-', alpha=0.1)
    xlim([-1,30])
    ylim([-30,30])
    ax = gca()
    ax.set_aspect(3/4)
    xlabel('Z (mm)')
    ylabel('Y (mm)')
    
    # create file name and append it to a list
    filename = f'{k}.png'
    filenames.append(filename)
    
    # save frame
    savefig(filename)
    close()# build gif
    
with imageio.get_writer('optimization_vs.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)

In [ ]:
figure()
semilogy(best_loss_log[20:])

In [ ]:
opt_i = tf.argmin(m._individual_training_losses())
print(m._individual_training_losses()[opt_i])
m.get_params(opt_i)

In [ ]:
opt_i = tf.argmin(m._individual_training_losses())
print(m._individual_training_losses()[opt_i])
ma = ARaycingModel(**{**m.get_params(opt_i)[0], 'energy': (m.energy1,), 'theta_min': -np.pi/2.2, 'theta_max': np.pi/2.2})
mb = ARaycingModel(**{**m.get_params(opt_i)[0], 'energy': (m.energy2,), 'theta_min': -np.pi/2.2, 'theta_max': np.pi/2.2})
figure(figsize=(5,5))
predicted_detpts1 = ma()
predicted_detpts2 = mb()
manifold_1 = get_manifold_from_pickle('../XRT/output_at_p=505_r=500_a=500_c=-500_e=9130.pickle', percentile=99)
manifold_2 = get_manifold_from_pickle('../XRT/output_at_p=505_r=500_a=500_c=-500_e=9140.pickle', percentile=99)
scatter(manifold_1[:,1], manifold_1[:,0],marker='.',c='k',label='xrt simulation at 9130 eV', alpha=0.5)
scatter(manifold_2[:,1], manifold_2[:,0],marker='.',c='g',label='xrt simulation at 9140 eV', alpha=0.5)
plot(predicted_detpts1[0,:,1],predicted_detpts1[0,:,0],'r-',label='optimized 9130 ev')
plot(predicted_detpts2[0,:,1],predicted_detpts2[0,:,0],'b-',label='optimized 9140 ev')
# plot(fit_line1[opt_i,:,1],fit_line1[opt_i,:,0],'r',label='9130 ev')
# plot(fit_line2[opt_i,:,1],fit_line2[opt_i,:,0],'b',label='9140 ev')
legend(loc=5)
xlim([-1,40])
ylim([-30,30])
ax = gca()
ax.set_aspect(3/4)
xlabel('Z (mm)')
ylabel('Y (mm)')
# savefig('vs_optimized.pdf')

In [ ]:
def initial_guesses_for_experimental_geometry2(N: int):
    grand = lambda x0, xu: np.random.uniform(low=x0-xu/2, high=x0+xu/2, size=(N,))
    f = 250
    c0 = -500
    a0 = 435
    b0 = 0.
    d0 = np.sqrt(a0**2 + c0**2)
    d_uncertainty = 10
    a_uncertainty = 10
    b_uncertainty = 10
    c_uncertainty = 10
    
    
    def ac_from_dc():
        d = grand(d0,d_uncertainty)
        c = grand(c0,c_uncertainty)
        return np.sqrt(d**2 - c**2), c
    a,c = ac_from_dc()
    b = grand(b0,b_uncertainty)
    
    p0 = 535
    q0 = 0.0
    r0 = 500
    q_uncertainty = 10
    r_uncertainty = 10
    p_uncertainty = 20
    
    p = grand(p0, p_uncertainty)
    r = grand(r0, r_uncertainty)
    q = grand(q0, q_uncertainty)
    
    t0 = 1
    u0 = 0
    v0 = 0
    t_uncertainty = 0.005
    u_uncertainty = 0.005
    v_uncertainty = 0.005
    
    t = grand(t0,t_uncertainty)
    u = grand(u0,u_uncertainty)
    v = grand(v0,v_uncertainty)
    T = change_of_basis(t,u,v)
    
    pqr = np.concatenate([p[:,None],q[:,None],r[:,None]],-1)
    pqr_prime = np.einsum('ijk,ik->ij',T,pqr)
    
    pp = pqr_prime[:,0]
    qp = pqr_prime[:,1]
    rp = pqr_prime[:,2]
    if N == 1:
        return {'a': float(a), 'b': float(b), 'c': float(c),
                'p': float(p), 'q': float(q), 'r': float(r),
                't': float(t), 'u': float(u), 'v': float(v)}
    else:
        return {'a': a, 'b': b, 'c': c, 'p': p, 'q': q, 'r': r, 't': t, 'u': u, 'v': v}

In [ ]:
initial_guess = initial_guesses_for_experimental_geometry2(250)
m2 = TwoLineFit(get_manifold_from_pickle('../XRT/output_at_p=550_r=500_a=450_c=-500_e=9130.pickle', percentile=99),
               get_manifold_from_pickle('../XRT/output_at_p=550_r=500_a=450_c=-500_e=9140.pickle', percentile=99),
               **{**initial_guess,
                                              'radius': 500,
                                              'energy1': 9130,
                                              'energy2': 9140,
                                              'mm_to_pixels': 1.,
                                              'xtal': CrystalSi(4,4,0),
                                              'theta_min': -np.pi/2.02,
                                              'theta_max': np.pi/2.02,
                                             })

opt_i0 = tf.argmin(m2._individual_training_losses())
opt_loss0 = m2._individual_training_losses()[opt_i0]
p0 = m2.get_params(opt_i0)
print('initial best guess loss: ', opt_loss0)

def plot_loss(loss_log, best_loss_log):
    figure(figsize=(10, 4))
    title('Loss history')
    semilogy(loss_log, 'k-.',alpha=0.3)
    figure(figsize=(10, 4))
    title('Best Loss history')
    semilogy(best_loss_log, 'r-.',alpha=0.9)
    show()
    
lr = 1E-2
# lr_sched = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
#     [1000, 2000, 3000, 4000, 5000], [lr, lr*0.5, lr*(0.5**2), lr*(0.5**3), lr*(0.5**4), lr*(0.5**5)])
lr_sched = tf.keras.experimental.CosineDecayRestarts(lr, 1000, alpha=0.1, m_mul=0.9, t_mul=1.0)
trainer = tf.keras.optimizers.Adam(lr_sched)

@tf.function
def train_step():
    with tf.GradientTape() as g:
        loss = m2.training_loss()
    grads = g.gradient(loss, m2.trainable_variables)
#     grads = [g/(tf.norm(g)+1e-8) for g in grads]
    trainer.apply_gradients(zip(grads, m2.trainable_variables))
    return loss

loss_log = []
best_loss_log = []
prediction_log = []
M = 100
for i in range(12000):
    loss = train_step()
    loss_log.append(loss.numpy())
    step_i = len(loss_log)
    if step_i%10 == 0:
        prediction_log.append(m2.raytracing_slice(m2.energy1, 20))
    if step_i%100 == 0:
        clear_output()
        indiv_losses = m2._individual_training_losses()
        best_loss_ind = tf.argmin(indiv_losses)
        best_loss_log.append(indiv_losses[best_loss_ind].numpy().item())
        plot_loss(loss_log, best_loss_log)
    if step_i%150 == 0:
        # prune the bad ones
        s = tf.argsort(m2._individual_training_losses()).numpy()
        # replace bad ones with scrambled versions of the top 20 particles
        ap = np.concatenate([m2.a.numpy()[s[:-M]], m2.a.numpy()[s[np.random.choice(M,M)]]],0)
        bp = np.concatenate([m2.b.numpy()[s[:-M]], m2.b.numpy()[s[np.random.choice(M,M)]]],0)
        cp = np.concatenate([m2.c.numpy()[s[:-M]], m2.c.numpy()[s[np.random.choice(M,M)]]],0)
        pp = np.concatenate([m2.p.numpy()[s[:-M]], m2.p.numpy()[s[np.random.choice(M,M)]]],0)
        qp = np.concatenate([m2.q.numpy()[s[:-M]], m2.q.numpy()[s[np.random.choice(M,M)]]],0)
        rp = np.concatenate([m2.r.numpy()[s[:-M]], m2.r.numpy()[s[np.random.choice(M,M)]]],0)
        tp = np.concatenate([m2.t.numpy()[s[:-M]], m2.t.numpy()[s[np.random.choice(M,M)]]],0)
        up = np.concatenate([m2.u.numpy()[s[:-M]], m2.u.numpy()[s[np.random.choice(M,M)]]],0)
        vp = np.concatenate([m2.v.numpy()[s[:-M]], m2.v.numpy()[s[np.random.choice(M,M)]]],0)
        m2.a.assign(ap)
        m2.b.assign(bp)
        m2.c.assign(cp)
        m2.p.assign(pp)
        m2.q.assign(qp)
        m2.r.assign(rp)
        m2.t.assign(tp)
        m2.u.assign(up)
        m2.v.assign(vp)
    print('\r step: %d, loss: %.3f'%(len(loss_log), loss), end='')

In [ ]:
figure()
semilogy(best_loss_log[20:])

In [ ]:
opt_i = tf.argmin(m2._individual_training_losses())
print(m2._individual_training_losses()[opt_i])
m2.get_params(opt_i)

In [ ]:
opt_i = tf.argmin(m2._individual_training_losses())
print(m2._individual_training_losses()[opt_i])
m2a = ARaycingModel(**{**m2.get_params(opt_i)[0], 'energy': (m2.energy1,), 'theta_min': -np.pi/2.02, 'theta_max': np.pi/2.02})
m2b = ARaycingModel(**{**m2.get_params(opt_i)[0], 'energy': (m2.energy2,), 'theta_min': -np.pi/2.02, 'theta_max': np.pi/2.02})
m2a0 = ARaycingModel(**{**p0[0], 'energy': (m2.energy1.numpy().item(),), 'theta_min': -np.pi/2.02, 'theta_max': np.pi/2.02})
m2b0 = ARaycingModel(**{**p0[1], 'energy': (m2.energy2.numpy().item(),), 'theta_min': -np.pi/2.02, 'theta_max': np.pi/2.02})
figure(figsize=(5,5))
predicted_detpts1 = m2a()
predicted_detpts2 = m2b()
predicted_detpts1_0 = m2a0()
predicted_detpts2_0 = m2b0()

manifold_1 = get_manifold_from_pickle('../XRT/output_at_p=550_r=500_a=450_c=-500_e=9130.pickle', percentile=99)
manifold_2 = get_manifold_from_pickle('../XRT/output_at_p=550_r=500_a=450_c=-500_e=9140.pickle', percentile=99)
scatter(manifold_1[:,1], manifold_1[:,0],marker='.',c='k',label='xrt simulation at 9130 eV', alpha=0.5)
scatter(manifold_2[:,1], manifold_2[:,0],marker='.',c='g',label='xrt simulation at 9140 eV', alpha=0.5)
plot(predicted_detpts1[0,:,1],predicted_detpts1[0,:,0],'r-',label='optimized 9130 ev')
plot(predicted_detpts2[0,:,1],predicted_detpts2[0,:,0],'b-',label='optimized 9140 ev')

legend(loc=5)
xlim([-15,30])
ylim([-10,10])
# xlim([-50,50])
# ylim([-50,50])
ax = gca()
ax.set_aspect(3/4)
xlabel('Z (mm)')
ylabel('Y (mm)')
# savefig('cartoid_optimized.pdf')

In [ ]:
preds = tf.stack(prediction_log,0)

In [ ]:
figure()
for k in range(300):
    plot(preds[k,0,:,1],preds[k,0,:,0],'k-',alpha=0.3)
xlim([-15,15])
ylim([-10,10])
# xlim([-50,50])
# ylim([-50,50])
ax = gca()
ax.set_aspect(3/4)
xlabel('Z (mm)')
ylabel('Y (mm)')

In [ ]:
preds = tf.stack(prediction_log,0)
filenames = []
for k in range(8,300):
    plot(preds[k,0,:,1],preds[k,0,:,0],'r-')
    plot(preds[k,1,:,1],preds[k,1,:,0],'k-', alpha=0.3)
    plot(preds[k,2,:,1],preds[k,2,:,0],'k-', alpha=0.3)
    plot(preds[k,3,:,1],preds[k,3,:,0],'k-', alpha=0.3)
    plot(preds[k,4,:,1],preds[k,4,:,0],'k-', alpha=0.3)
    plot(preds[k,5,:,1],preds[k,5,:,0],'k-', alpha=0.3)
    plot(preds[k,6,:,1],preds[k,6,:,0],'k-', alpha=0.3)
    plot(preds[k,7,:,1],preds[k,7,:,0],'k-', alpha=0.2)
    plot(preds[k,8,:,1],preds[k,8,:,0],'k-', alpha=0.2)
    plot(preds[k,9,:,1],preds[k,9,:,0],'k-', alpha=0.2)
    plot(preds[k,10,:,1],preds[k,10,:,0],'k-', alpha=0.2)
    plot(preds[k,11,:,1],preds[k,11,:,0],'k-', alpha=0.2)
    plot(preds[k,12,:,1],preds[k,12,:,0],'k-', alpha=0.2)
    plot(preds[k,13,:,1],preds[k,13,:,0],'k-', alpha=0.1)
    plot(preds[k,14,:,1],preds[k,14,:,0],'k-', alpha=0.1)
    plot(preds[k,15,:,1],preds[k,15,:,0],'k-', alpha=0.1)
    plot(preds[k,16,:,1],preds[k,16,:,0],'k-', alpha=0.1)
    plot(preds[k,17,:,1],preds[k,17,:,0],'k-', alpha=0.1)
    plot(preds[k,18,:,1],preds[k,18,:,0],'k-', alpha=0.1)
    plot(preds[k,19,:,1],preds[k,19,:,0],'k-', alpha=0.1)
    xlim([-15,15])
    ylim([-10,10])
    ax = gca()
    ax.set_aspect(3/4)
    xlabel('Z (mm)')
    ylabel('Y (mm)')
    
    # create file name and append it to a list
    filename = f'{k}.png'
    filenames.append(filename)
    
    # save frame
    savefig(filename)
    close()# build gif
    
with imageio.get_writer('optimization_cartoid.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)